먼저 Feature들을 하나씩 만든다.
그리고 xgboost를 강의 내용을 따라하면서 튜닝해서 만들어본다.

https://github.com/KaggleBreak/walkingkaggle/blob/master/rental_listing/rental_listing_0422.ipynb

## Features & Issues

list all features
  * Numerical data
    * price
    * price_per_bedrooms: price / bedrooms
    * bathrooms
    * bedrooms
    * room_sum: bedrooms + bathrooms
    * num_priceXroom: price / (1 + bedrooms + 0.5*bathrooms) -> https://www.kaggle.com/arnaldcat/two-sigma-connect-rental-listing-inquiries/a-proxy-for-sqft-and-the-interest-on-1-2-baths 참고했다고 한다. 
    * latitude
    * longitude
    * dist_from_center: sqrt((latitude-latitude.median())**2 + (longitude - longitude.median()) & apply precision 
    * num_photos: apply len to photos list
    * num_features: apply len to features list
    * num_description_words: apply split(" ") to description
    * building_avg_score
    * building_frequency
    * manager_avg_score
    * manager_frequency
    * created: Neural Network 파트에서 weekday, weekofyear로 변환해서 사용. **시간도 의미가 있었는데** 이걸 사용할 필요성 있을듯?
  * Categorical data
    * display_address
   
    
raw data but not used for feature
  * listing_id -> 이건 created 별로 scaling을 해서 쓰는게 맞다고 보는데. categorical로 할 필요가 없을 듯.
  * manager_id, building_id는 각각 평균 score와 빈도로 numerical data로 바꿨는데 또 추가할 필요가 있을까?
  * steet_address는 할 필요 없고 display_address로만 하는것이 좋을 듯. 동네만 알면 되지 번지까지 할 필욘 없으니깐.

Features related issues
  * interest_level의 대부분이 low이고 high가 적기 때문에 클래스간 비율을 위해 oversampling / undersampling이 필요할 듯
  * interest_level의 Numerical incoding이 필요할까? -> low, medium, high이기 때문에 0, 1, 2를 적용하는 것은 합리적이긴 함.
  * tfidf 팩터화..가 뭘까? 뭔가 stop words를 제외시키는 작업인 듯 한데.
  * outlier들: price, bedrooms, bathrooms, longitude, latitude 처리 필요성 -> pandas의 clip을 사용하면 아웃라이어들을 특정한 값으로 지정 가능.
  * dist_from_center의 precision을 적용, 일정구역을 같은 값으로 설정한 후에 density도 구하던데, 이게 feature로 유용할지 잘 모르겠어서 일단 제외.
  * Neural Network 파트에서 building_id와 manager_id에 대해 supervised를 계산하는데 이게 뭔지 잘 모르겠음. frequency는 알것는데. 이거 확인해보고 feature에 넣어주는 걸 고려
  * Neural Network 파트에서 price를 digitize를 한다. 뭐하는건지 확인.
  * Neural Network 파트에서 0.5 단위 bathrooms를 num_even_bathrooms로 처리를 해주는데 어떻게 넣어줬는지 잘 모르겠음.
  * Neural Network 파트에서 특정 단어들에 대해서 몇번 나왔는지 체크하는 것도 feature로 사용한듯 보인다. 

XGBoost related issues
  * change data to fit on XGB
  * XGB는 아웃라이어에 대해 영향을 덜 받을까?

sklearn에서는 DataFrameMapper로 StandardScaler를 사용할 수 있는 듯. -> 이거 볼 필요성은 있겠다. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_json('./data/train.json')
test = pd.read_json('./data/test.json')

In [4]:
train.tail()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
9999,1.0,2,297160caf8a88c10718b492d4672c6b4,2016-06-02 05:41:05,"30TH/3RD, MASSIVE CONV 2BR IN LUXURY FULL SERV...",E 30 St,"[Elevator, Laundry in Building, Laundry in Uni...",medium,40.7426,7098690,-73.9790,2a1ee03b449700c3a15dd8c9a505c525,[https://photos.renthop.com/2/7098690_18396d32...,3200,230 E 30 St
99991,1.0,1,9e871ccfea7b07686c101548ca693af6,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",low,40.7102,6822449,-74.0163,ef20d40b528500037e37b167787ac089,[https://photos.renthop.com/2/6822449_b429587b...,3950,225 Rector Place
99992,1.0,1,7967a1280bf3f7644500fc79d2696b0e,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,"[Doorman, Elevator, Pre-War, Dogs Allowed, Cat...",low,40.7601,6881461,-73.9900,d889a9552265fe359e7d36d172084fcb,[https://photos.renthop.com/2/6881461_20a86530...,2595,341 West 45th Street
99993,1.0,0,ad67f6181a49bde19218929b401b31b7,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,"[Doorman, Elevator, Pre-War, Dogs Allowed, Cat...",low,40.7066,6841891,-74.0101,9fd3af5b2d23951e028059e8940a55d7,[https://photos.renthop.com/2/6841891_124c9c44...,3350,37 Wall Street
99994,1.0,2,5173052db6efc0caaa4d817112a70f32,2016-04-12 02:48:07,Look no further!!! This giant 2 bedroom apart...,Park Terrace East,[Hardwood Floors],low,40.8699,6858245,-73.9172,d7f57128272bfd82e33a61999b5f4c42,[https://photos.renthop.com/2/6858245_c4380bde...,2200,30 Park Terrace East


### Numerical Features

* price
* price_per_bedrooms: price / bedrooms
* bathrooms
* bedrooms
* room_sum: bedrooms + bathrooms
* num_priceXroom: price / (1 + bedrooms + 0.5*bathrooms) -> https://www.kaggle.com/arnaldcat/two-sigma-connect-rental-listing-inquiries/a-proxy-for-sqft-and-the-interest-on-1-2-baths 참고했다고 한다. 
* latitude
* longitude
* dist_from_center: sqrt((latitude-latitude.median())**2 + (longitude - longitude.median()) & apply precision 
* num_photos: apply len to photos list
* num_features: apply len to features list
* num_description_words: apply split(" ") to description
* building_avg_score
* building_frequency
* manager_avg_score
* manager_frequency
* created: Neural Network 파트에서 weekday, weekofyear로 변환해서 사용.

In [5]:
# numerical encoding of target value
train.loc[:, 'interest_level_in_num'] = 0.0
train.loc[train.loc[:, 'interest_level'] == 'medium', 'interest_level_in_num'] = 1.0
train.loc[train.loc[:, 'interest_level'] == 'high', 'interest_level_in_num'] = 2.0

# making derivated features for train data
train.loc[:, 'price_per_bedrooms'] = train.loc[:, 'price'] / train.loc[:, 'bedrooms']
# https://www.kaggle.com/arnaldcat/two-sigma-connect-rental-listing-inquiries/a-proxy-for-sqft-and-the-interest-on-1-2-baths
train.loc[:, 'priceXroom'] = (train.loc[:, 'price'] / (1. + train.loc[:, 'bedrooms'].clip(1, 4) + \
                                                       0.5*train.loc[:, 'bathrooms'].clip(0, 2)))
train.loc[:, 'room_sum'] = train.loc[:, 'bedrooms'] + train.loc[:, 'bathrooms']
train.loc[:, 'dist_from_center'] = np.sqrt((train.loc[:, 'latitude'] - train.loc[:, 'latitude'].median())**2 + \
                                           (train.loc[:, 'longitude'] - train.loc[:, 'longitude'].median())**2)
train.loc[:, 'num_photos'] = train.loc[:, 'photos'].apply(len)
train.loc[:, 'num_features'] = train.loc[:, 'features'].apply(len)
# maybe better to exclude 'stop words' by using tfid??
train.loc[:, 'num_description_words'] = train.loc[:, 'description'].apply(lambda x: len(x.split(" ")))

# apply scores depends on building id and manager id
b_id_freq_th = 15. # frequency threshold for assigning building id interest level in num score
m_id_freq_th = 15. # frequency threshold for assigning manager id interest level in num score

grouped = train.groupby('building_id')['interest_level_in_num'].aggregate(['count', 'mean'])
tmp_dic = grouped.loc[grouped.loc[:, 'count'] > b_id_freq_th, 'mean'].to_dict()
tmp_avg = train.loc[:, 'interest_level_in_num'].mean()
train.loc[:, 'building_avg_score'] = train.loc[:, 'building_id'].apply(lambda x: tmp_dic.get(x, tmp_avg))
test.loc[:, 'building_avg_score'] = test.loc[:, 'building_id'].apply(lambda x: tmp_dic.get(x, tmp_avg)) # using train data

tmp_dic = train.loc[:, 'building_id'].value_counts().to_dict()
tmp_avg = train.loc[:, 'building_id'].value_counts().min()
train.loc[:, 'building_frequency'] = train.loc[:, 'building_id'].apply(lambda x: tmp_dic.get(x, tmp_avg))
test.loc[:, 'building_frequency'] = test.loc[:, 'building_id'].apply(lambda x: tmp_dic.get(x, tmp_avg)) # using train data

grouped = train.groupby('manager_id')['interest_level_in_num'].aggregate(['count', 'mean'])
tmp_dic = grouped.loc[grouped.loc[:, 'count'] > m_id_freq_th, 'mean'].to_dict()
tmp_avg = train.loc[:, 'interest_level_in_num'].mean()
train.loc[:, 'manager_avg_score'] = train.loc[:, 'manager_id'].apply(lambda x: tmp_dic.get(x, tmp_avg))
test.loc[:, 'manager_avg_score'] = test.loc[:, 'manager_id'].apply(lambda x: tmp_dic.get(x, tmp_avg)) # using train data

tmp_dic = train.loc[:, 'manager_id'].value_counts().to_dict()
tmp_avg = train.loc[:, 'manager_id'].value_counts().min()
train.loc[:, 'manager_frequency'] = train.loc[:, 'manager_id'].apply(lambda x: tmp_dic.get(x, tmp_avg))
test.loc[:, 'manager_frequency'] = test.loc[:, 'manager_id'].apply(lambda x: tmp_dic.get(x, tmp_avg)) # using train data

# created
train.loc[:, 'created'] = pd.to_datetime(train.loc[:, 'created'])
train.loc[:, 'dayofweek'] = train.loc[:, 'created'].dt.dayofweek
train.loc[:, 'weekofyear'] = train.loc[:, 'created'].dt.weekofyear

In [6]:
# making derivated features for test data
test.loc[:, 'price_per_bedrooms'] = test.loc[:, 'price'] / test.loc[:, 'bedrooms']
# https://www.kaggle.com/arnaldcat/two-sigma-connect-rental-listing-inquiries/a-proxy-for-sqft-and-the-interest-on-1-2-baths
test.loc[:, 'priceXroom'] = (test.loc[:, 'price'] / (1. + test.loc[:, 'bedrooms'].clip(1, 4) + \
                                                       0.5*test.loc[:, 'bathrooms'].clip(0, 2)))
test.loc[:, 'room_sum'] = test.loc[:, 'bedrooms'] + test.loc[:, 'bathrooms']
test.loc[:, 'dist_from_center'] = np.sqrt((test.loc[:, 'latitude'] - test.loc[:, 'latitude'].median())**2 + \
                                           (test.loc[:, 'longitude'] - test.loc[:, 'longitude'].median())**2)
test.loc[:, 'num_photos'] = test.loc[:, 'photos'].apply(len)
test.loc[:, 'num_features'] = test.loc[:, 'features'].apply(len)
# maybe better to exclude 'stop words' by using tfid??
test.loc[:, 'num_description_words'] = test.loc[:, 'description'].apply(lambda x: len(x.split(" ")))

# created
test.loc[:, 'created'] = pd.to_datetime(test.loc[:, 'created'])
test.loc[:, 'dayofweek'] = test.loc[:, 'created'].dt.dayofweek
test.loc[:, 'weekofyear'] = test.loc[:, 'created'].dt.weekofyear

### Categorical Features

* display_address

제외
* listing_id -> 이건 created 별로 scaling을 해서 쓰는게 맞다고 보는데. categorical로 할 필요가 없을 듯.
* manager_id, building_id는 각각 평균 score와 빈도로 numerical data로 바꿨는데 또 추가할 필요가 있을까?
* steet_address는 할 필요 없고 display_address로만 하는것이 좋을 듯. 동네만 알면 되지 번지까지 할 필욘 없으니깐.

Issues
* 근데 encoding을 LabelEncoder로 하는게 맞나? OneHotEncoder로 해야 맞는거 아닌가?

In [7]:
from sklearn import preprocessing

In [8]:
le = preprocessing.LabelEncoder()
le.fit(np.append(train.loc[:, 'display_address'].values, test.loc[:, 'display_address'].values))
train.loc[:, 'le_display_address'] = le.transform(train.loc[:, 'display_address'].values)
test.loc[:, 'le_display_address'] = le.transform(test.loc[:, 'display_address'].values)

In [9]:
train.loc[:, 'le_display_address'].tail()

9999      9577
99991    12864
99992    15306
99993    14850
99994    12707
Name: le_display_address, dtype: int64

In [10]:
features_to_use = ['price', 'price_per_bedrooms', 'priceXroom', 'bathrooms', 'bedrooms', 'room_sum', 'latitude', \
                   'longitude', 'num_photos', 'num_features', 'num_description_words', 'building_avg_score', \
                   'building_frequency', 'manager_avg_score', 'manager_frequency', 'dayofweek', 'weekofyear', \
                   'le_display_address']

In [11]:
len(features_to_use)

18

## xgb part

### data transform for xgb

In [14]:
import xgboost as xgb

In [15]:
# https://datascience.stackexchange.com/questions/12799/pandas-dataframe-to-dmatrix
xgtrain = xgb.DMatrix(train.loc[:, features_to_use].values, label=train.loc[:, 'interest_level_in_num'].values)

### Running xgb

In [16]:
params = {
    'objective':'multi:softprob',
    'max_depth':2,
    'silent':1,
    'eta':1,
    'num_class':3,
    'eval_metric':'mlogloss'
}

num_rounds = 5

In [17]:
model = xgb.train(params, xgtrain, num_rounds)